In [1]:
import sys
sys.path.append("../")

In [3]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

# Data Filtering Example for FPA Auction Data

## Overview

This notebook demonstrates a suggested approach for filtering data from the FPA (First-Price Auction) data files. The methods shown here are intended as guidelines and can be adapted based on specific requirements.

## Purpose

The main objectives of this notebook are:

1. To showcase a potential data filtering methodology
2. To provide a starting point for working with FPA auction data
3. To replicate the data preprocessing steps applied to the algorithms presented in the benchmark's accompanying research paper, ensuring consistency with the published results

## Important Note

While this notebook presents one way to filter and process the data, it's important to remember that:

- This is not the only correct method
- The approach can and should be modified to suit specific research needs
- Users are encouraged to explore alternative filtering techniques that may be more appropriate for their particular analysis

### Stats and campaigns loading separately

In [14]:
stats = pd.read_csv('../data/fpa/stats.csv')
campaigns = pd.read_csv('../data/fpa/campaigns.csv')

In [19]:
stats.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3668 entries, 1518 to 109232
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   item_id                   3668 non-null   int64  
 1   campaign_id               3668 non-null   int64  
 2   period                    3668 non-null   float64
 3   contact_price_bin         3668 non-null   int64  
 4   AuctionVisibilitySurplus  3668 non-null   float64
 5   AuctionClicksSurplus      3668 non-null   float64
 6   AuctionContactsSurplus    3668 non-null   float64
 7   AuctionWinBidSurplus      3668 non-null   float64
 8   CTRPredicts               3668 non-null   float64
 9   CRPredicts                3668 non-null   float64
 10  AuctionCount              3668 non-null   float64
dtypes: float64(8), int64(3)
memory usage: 343.9 KB


In [ ]:
def process_data(campaigns, stats):
    # Filter out invalid or undesirable data points
    stats = stats[
        stats['contact_price_bin'].notna() &
        (stats['AuctionVisibilitySurplus'] > 0) &
        (stats['AuctionClicksSurplus'] > 0) &
        (stats['AuctionContactsSurplus'] > 0) &
        (stats['AuctionWinBidSurplus'] > 0) &
        (stats['CTRPredicts'] > 0) &
        (stats['CRPredicts'] > 0)
    ]

    # Aggregate stats by campaign
    cmp_stats = stats.groupby('campaign_id').agg({
        'period': ['nunique', 'max', 'min'],
        'AuctionWinBidSurplus': 'sum',
        'AuctionContactsSurplus': 'sum',
        'AuctionClicksSurplus': 'sum',
        'contact_price_bin': 'max'
    }).reset_index()

    # Rename columns for clarity
    cmp_stats.columns = ['campaign_id', 'count_periods', 'max_period', 'min_period', 
                         'max_price_vcg', 'AuctionContactsSurplus', 'max_clicks', 'max_contact_price_bin']

    # Calculate FPA max price and max contacts
    cmp_stats['max_price_fpa'] = cmp_stats['AuctionContactsSurplus'] * np.power(1.2, cmp_stats['max_contact_price_bin'])
    cmp_stats['max_contacts'] = cmp_stats['AuctionContactsSurplus']
    cmp_stats.drop('AuctionContactsSurplus', axis=1, inplace=True)

    # Calculate number of periods and check if all periods are logged
    cmp_stats['num_periods'] = ((cmp_stats['max_period'] - cmp_stats['min_period'])/3600 + 1).astype(int)
    cmp_stats['period_full'] = cmp_stats['num_periods'] == cmp_stats['count_periods']

    # Filter and process campaigns data
    campaigns = campaigns[((campaigns['campaign_end'] - campaigns['campaign_start'])/3600/24) > 0.99]
    campaigns['count_c'] = campaigns.groupby('campaign_id')['campaign_id'].transform('count')
    campaigns = campaigns[campaigns['count_c'] == 1]
    campaigns = campaigns[['campaign_id', 'campaign_end_date', 'campaign_start', 'campaign_end', 'auction_budget']]

    # Merge campaign data with stats
    cmp_stats = pd.merge(cmp_stats, campaigns, on='campaign_id', how='inner')

    # Check if logs start at campaign start
    cmp_stats['logs_at_start'] = cmp_stats['min_period'] <= cmp_stats['campaign_start']

    # Calculate various metrics and flags
    cmp_stats['expected_periods'] = ((cmp_stats['campaign_end'] - cmp_stats['campaign_start'])/3600).astype(int)
    cmp_stats['vcg_campaign_days'] = ((cmp_stats['campaign_end'] - cmp_stats['campaign_start'])/3600/24).astype(int)
    cmp_stats['periods_logged_full'] = cmp_stats['count_periods'] >= cmp_stats['expected_periods'] - 1
    cmp_stats['budget_vcg_ok'] = cmp_stats['max_price_vcg'] > cmp_stats['auction_budget']
    cmp_stats['budget_fpa_ok'] = cmp_stats['max_price_fpa'] > cmp_stats['auction_budget']
    cmp_stats['clicks_ok'] = cmp_stats['max_clicks'] > 3
    cmp_stats['contacts_ok'] = cmp_stats['max_contacts'] > 0.5
    cmp_stats['campaign>1day'] = cmp_stats['expected_periods'] > 25
    cmp_stats['campaign>=7day'] = cmp_stats['expected_periods'] > 24 * 7
    cmp_stats['logged>1day'] = cmp_stats['num_periods'] > 25
    cmp_stats['logged>2day'] = cmp_stats['num_periods'] > 49

    campaigns_tier_1 = cmp_stats[
        cmp_stats['budget_vcg_ok'] &
        cmp_stats['budget_fpa_ok'] &
        cmp_stats['clicks_ok'] &
        cmp_stats['logs_at_start']
    ]

    return cmp_stats, campaigns_tier_1

In [ ]:
cmp_stats, campaigns_tier_1 = process_data(campaigns, stats)

In [ ]:
ministats = stats[stats.campaign_id.isin(set(campaigns_tier_1.campaign_id))]
minicampaigns = campaigns[campaigns.campaign_id.isin(set(campaigns_tier_1.campaign_id))]

In [ ]:
len(minicampaigns), len(campaigns)

In [ ]:
minicampaigns

# Some statistics of the datasets

In [ ]:
# Contact price bin distribution
stats.contact_price_bin.hist(bins=30)

In [ ]:
def campaigns_value_counts(campaigns: pd.DataFrame):
    length_series = pd.to_datetime(campaigns['campaign_end'], unit='s') - pd.to_datetime(campaigns['campaign_start'], unit='s')
    print(length_series.value_counts())
    pass

In [ ]:
# Distribution of the lengths of the lifetime of the campaigns
campaigns_value_counts(campaigns)

# Train + test split

In [ ]:
minicampaigns.campaign_start_date.value_counts()

In [ ]:
k = len(minicampaigns) * 2 // 15
k_test = len(minicampaigns) * 4 // 15
last_train_start = max(np.partition(minicampaigns.campaign_start, k)[:k])
last_test_start = max(np.partition(minicampaigns.campaign_start, k_test)[:k_test])
print(f'length of the train: {k}, length of the test: {k_test - k}')

In [ ]:
len(minicampaigns[minicampaigns.campaign_start < last_train_start]), len(minicampaigns[(minicampaigns.campaign_start >= last_train_start) & (minicampaigns.campaign_start < last_test_start)])

In [ ]:
# Filter out campaigns from the training set whose lifetimes intersect with any campaign in the test set
campaigns_train = minicampaigns[minicampaigns.campaign_start < last_train_start]
campaigns_test = minicampaigns[(minicampaigns.campaign_start >= last_train_start) & (minicampaigns.campaign_start < last_test_start)]

# Saving filtered data to files

In [ ]:
minicampaigns.to_csv('../data/fpa/campaigns_fpa_filtered_final.csv', index=False)
campaigns_train.to_csv('../data/fpa/campaigns_fpa_filtered_train_final.csv', index=False)
campaigns_test.to_csv('../data/fpa/campaigns_fpa_filtered_test_final.csv', index=False)

In [ ]:
ministats.to_csv('../data/fpa/stats_fpa_filtered_final.csv', index=False)
ministats[ministats.campaign_id.isin(campaigns_train.campaign_id)].to_csv('../data/fpa/stats_fpa_filtered_train_final.csv', index=False)
ministats[ministats.campaign_id.isin(campaigns_test.campaign_id)].to_csv('../data/fpa/stats_fpa_filtered_test_final.csv', index=False)